In [ ]:
import re
import json
import ast
import glob
import pandas as pd
import numpy as np
from typing import List, Tuple, Union, Dict, Any
from concurrent.futures import ThreadPoolExecutor
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from sentence_transformers import SentenceTransformer
from kiwipiepy import Kiwi
from rapidfuzz import fuzz, process
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import HtmlFormatter
from IPython.display import HTML
import matplotlib.pyplot as plt
from difflib import SequenceMatcher
import torch

# Configuration
pd.set_option('display.max_colwidth', 500)

# API Configuration
try:
    import config
    LLM_API_KEY = config.CUSTOM_API_KEY
    LLM_API_URL = "https://api.platform.a15t.com/v1"
except ImportError:
    LLM_API_KEY = "sktax-z2GВxrBfqЗР71Хh7RNIst3xchZdoOjyixfyHlIzGttXuz"
    LLM_API_URL = "https://apigw.sktax.chat/api/v1/serving/openai-api/v1"

# Initialize OpenAI Clients
client = OpenAI(api_key=LLM_API_KEY, base_url=LLM_API_URL)
client_2 = OpenAI(
    api_key="sktax-z2GВxrBfqЗР71Хh7RNIst3xchZdoOjyixfyHlIzGttXuz",
    base_url="https://apigw.sktax.chat/api/v1/serving/openai-api/v1"
)

# Initialize Language Models
def initialize_llm(model: str = "skt/claude-3-7-sonnet-20250219", max_tokens: int = 100) -> ChatOpenAI:
    """Initialize a language model with specified parameters."""
    return ChatOpenAI(
        temperature=0,
        openai_api_key=LLM_API_KEY,
        openai_api_base=LLM_API_URL,
        model=model,
        max_tokens=max_tokens
    )

llm_cld37 = initialize_llm()
llm_gem3 = initialize_llm(model="skt/gemma3-12b-it")
llm_chat = ChatOpenAI(
    temperature=0,
    model="gpt-4o",
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    max_tokens=2000
)
llm_cld40 = ChatAnthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY"),
    model="claude-sonnet-4-20250514",
    max_tokens=3000
)

# Data Loading
mms_pdf = pd.read_csv("./data/mms_data_250408.csv")
mms_pdf['msg'] = mms_pdf['msg_nm'] + "\n" + mms_pdf['mms_phrs']
mms_pdf = mms_pdf.groupby(["msg_nm", "mms_phrs", "msg"])['offer_dt'].min().reset_index(name="offer_dt")
mms_pdf = mms_pdf.reset_index().astype(str)

item_pdf_raw = pd.read_csv("./data/item_info_all_250527.csv")
item_pdf_all = item_pdf_raw.drop_duplicates(['item_nm', 'item_id'])[
    ['item_nm', 'item_id', 'item_desc', 'domain', 'start_dt', 'end_dt', 'rank']
].copy()

# Alias Rules and Entity Extension
alia_rule_set = list(zip(
    pd.read_csv("./data/alias_rules.csv")['alias_1'],
    pd.read_csv("./data/alias_rules.csv")['alias_2']
))

def apply_alias_rule(item_nm: str) -> List[str]:
    """Apply alias rules to generate alternative item names."""
    item_nm_list = [item_nm]
    for alias_1, alias_2 in alia_rule_set:
        if alias_1 in item_nm:
            item_nm_list.append(item_nm.replace(alias_1, alias_2))
        if alias_2 in item_nm:
            item_nm_list.append(item_nm.replace(alias_2, alias_1))
    return item_nm_list

item_pdf_all['item_nm_alias'] = item_pdf_all['item_nm'].apply(apply_alias_rule)
item_pdf_all = item_pdf_all.explode('item_nm_alias')

# User-Defined Entities
user_defined_entity = ['AIA Vitality', '부스트 파크 건대입구', 'Boost Park �대입구']
item_pdf_ext = pd.DataFrame([
    {'item_nm': e, 'item_id': e, 'item_desc': e, 'domain': 'user_defined', 
     'start_dt': 20250101, 'end_dt': 99991231, 'rank': 1, 'item_nm_alias': e}
    for e in user_defined_entity
])
item_pdf_all = pd.concat([item_pdf_all, item_pdf_ext])

# Entity List for Fuzzy Matching
entity_list_for_fuzzy = [
    (row['item_nm'], {
        'item_id': row['item_id'], 'description': row['item_desc'], 'domain': row['domain'],
        'start_dt': row['start_dt'], 'end_dt': row['end_dt'], 'rank': 1, 'item_nm_alias': row['item_nm_alias']
    })
    for row in item_pdf_all.to_dict('records')
]

# Stop Words and Sentence Transformer
stop_item_names = pd.read_csv("./data/stop_words.csv")['stop_words'].to_list()
model = SentenceTransformer('jhgan/ko-sbert-nli')
pgm_pdf = pd.read_csv("./data/pgm_tag_ext_250516.csv")
clue_embeddings = model.encode(
    pgm_pdf[["pgm_nm", "clue_tag"]].apply(
        lambda x: preprocess_text(x['pgm_nm'].lower()) + " " + x['clue_tag'].lower(), axis=1
    ).tolist(),
    convert_to_tensor=True
)

# Text Processing Utilities
def preprocess_text(text: str) -> str:
    """Preprocess text by removing special characters and normalizing spaces."""
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def clean_text(text: str) -> str:
    """Clean text while preserving JSON structure and Korean characters."""
    placeholders = {
        '"': "DQUOTE_TOKEN", "'": "SQUOTE_TOKEN", "{": "OCURLY_TOKEN", "}": "CCURLY_TOKEN",
        "[": "OSQUARE_TOKEN", "]": "CSQUARE_TOKEN", ":": "COLON_TOKEN", ",": "COMMA_TOKEN"
    }
    for char, placeholder in placeholders.items():
        text = text.replace(char, placeholder)
    
    text = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F\x7F]', '', text)
    text = re.sub(r'\r\n|\r', '\n', text)
    text = re.sub(r'[\u200B-\u200D\uFEFF\u00A0]', '', text)
    allowed_chars = (
        r'[^\x00-\x7F\u0080-\u00FF\u0100-\u024F\u0370-\u03FF\u0400-\u04FF'
        r'\u1100-\u11FF\u3130-\u318F\uA960-\uA97F\u3000-\u303F'
        r'\uAC00-\uD7A3\uFF00-\uFFEF\u4E00-\u9FFF\n\r\t ]'
    )
    text = re.sub(allowed_chars, '', text)
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r'\n\s*\n+', '\n\n', text)
    
    for char, placeholder in placeholders.items():
        text = text.replace(placeholder, char)
    
    text = re.sub(r'"\s+:', r'":', text)
    text = re.sub(r',\s*]', r']', text)
    text = re.sub(r',\s*}', r'}', text)
    return text

def repair_json(broken_json: str) -> str:
    """Repair ill-formed JSON strings."""
    json_str = re.sub(r':\s*([a-zA-Z0-9_]+)(\s*[,}])', r': "\1"\2', broken_json)
    json_str = re.sub(r'([{,])\s*([a-zA-Z0-9_]+):', r'\1 "\2":', json_str)
    json_str = re.sub(r',\s*}', '}', json_str)
    return json_str

def extract_json_objects(text: str) -> List[Dict]:
    """Extract valid JSON objects from text."""
    pattern = r'(\{(?:[^{}]|(?:\{(?:[^{}]|(?:\{[^{}]*\}))*\}))*\})'
    result = []
    for match in re.finditer(pattern, text):
        potential_json = match.group(0)
        try:
            json_obj = ast.literal_eval(clean_ill_structured_json(repair_json(potential_json)))
            result.append(json_obj)
        except (json.JSONDecodeError, SyntaxError):
            pass
    return result

# Entity Matching with Kiwi
kiwi = Kiwi()
kiwi_raw = Kiwi()
kiwi_raw.space_tolerance = 2
stop_item_names = list(set(stop_item_names + [x.lower() for x in stop_item_names]))
entity_list_for_kiwi = list(item_pdf_all['item_nm_alias'].unique())
for w in entity_list_for_kiwi:
    kiwi.add_user_word(w, "NNP")
for w in stop_item_names:
    kiwi.add_user_word(w, "NNG")

tags_to_exclude = ['W_SERIAL', 'W_URL', 'JKO', 'SSO', 'SSC', 'SW', 'SF', 'SP', 'SS', 'SE', 'SO', 'SB', 'SH']
exc_tag_patterns = [
    ['SN', 'NNB'], ['W_SERIAL'], ['JKO'], ['W_URL'], ['W_EMAIL'],
    ['XSV', 'EC'], ['VV', 'EC'], ['VCP', 'ETM'], ['XSA', 'ETM'], ['VV', 'ETN']
] + [[t] for t in tags_to_exclude]

class Token:
    def __init__(self, form: str, tag: str, start: int, len: int):
        self.form = form
        self.tag = tag
        self.start = start
        self.len = len
    
    def __repr__(self) -> str:
        return f"Token(form='{self.form}', tag='{self.tag}', start={self.start}, len={self.len})"

edf = item_pdf_all.copy()
edf['token_entity'] = edf.apply(
    lambda x: [d[0] for d in kiwi_raw.tokenize(x['item_nm_alias'], normalize_coda=True, z_coda=False, split_complex=False) 
               if d[1] not in tags_to_exclude], axis=1
)
edf['char_entity'] = edf.apply(lambda x: list(x['item_nm_alias'].lower().replace(' ', '')), axis=1)

# Korean Entity Matching
class KoreanEntityMatcher:
    def __init__(self, min_similarity: int = 70, ngram_size: int = 2, min_entity_length: int = 2, token_similarity: bool = True):
        self.min_similarity = min_similarity
        self.ngram_size = ngram_size
        self.min_entity_length = min_entity_length
        self.token_similarity = token_similarity
        self.entities = []
        self.entity_data = {}
        self.ngram_index = {}
        self.normalized_entities = {}

    def build_from_list(self, entities: List[Union[str, Tuple[str, Dict]]]):
        """Build entity index from a list of entities."""
        self.entities = []
        self.entity_data = {}
        for i, entity in enumerate(entities):
            if isinstance(entity, tuple) and len(entity) == 2:
                entity_name, data = entity
                self.entities.append(entity_name)
                self.entity_data[entity_name] = data
            else:
                self.entities.append(entity)
                self.entity_data[entity] = {'id': i, 'entity': entity}
        
        for entity in self.entities:
            normalized = self._normalize_text(entity)
            self.normalized_entities[normalized] = entity
        self._build_ngram_index(n=self.ngram_size)

    def _normalize_text(self, text: str) -> str:
        """Normalize text by converting to lowercase and unifying spaces."""
        text = text.lower()
        text = re.sub(r'\s+', ' ', text)
        return text.strip()

    def _tokenize(self, text: str) -> List[str]:
        """Tokenize text into Korean, English, and numeric tokens."""
        return re.findall(r'[가-힣]+|[a-z0-9]+', self._normalize_text(text))

    def _build_ngram_index(self, n: int):
        """Build n-gram index optimized for Korean characters."""
        for entity in self.entities:
            if len(entity) < self.min_entity_length:
                continue
            normalized_entity = self._normalize_text(entity)
            entity_chars = list(normalized_entity)
            ngrams = [''.join(entity_chars[i:i+n]) for i in range(len(entity_chars) - n + 1)]
            for ngram in ngrams:
                if ngram not in self.ngram_index:
                    self.ngram_index[ngram] = set()
                self.ngram_index[ngram].add(entity)
            
            tokens = self._tokenize(normalized_entity)
            for token in tokens:
                if len(token) >= n:
                    token_key = f"TOKEN:{token}"
                    if token_key not in self.ngram_index:
                        self.ngram_index[token_key] = set()
                    self.ngram_index[token_key].add(entity)

    def _get_candidates(self, text: str, n: int = None) -> List[Tuple[str, float]]:
        """Get candidate entities based on n-gram overlap."""
        if n is None:
            n = self.ngram_size
        normalized_text = self._normalize_text(text)
        if normalized_text in self.normalized_entities:
            return [(self.normalized_entities[normalized_text], float('inf'))]
        
        text_chars = list(normalized_text)
        text_ngrams = set(''.join(text_chars[i:i+n]) for i in range(len(text_chars) - n + 1))
        tokens = self._tokenize(normalized_text)
        for token in tokens:
            if len(token) >= n:
                text_ngrams.add(f"TOKEN:{token}")
        
        candidates = set()
        for ngram in text_ngrams:
            if ngram in self.ngram_index:
                candidates.update(self.ngram_index[ngram])
        
        candidate_scores = {}
        for candidate in candidates:
            candidate_normalized = self._normalize_text(candidate)
            candidate_chars = list(candidate_normalized)
            candidate_ngrams = set(''.join(candidate_chars[i:i+n]) for i in range(len(candidate_chars) - n + 1))
            candidate_tokens = self._tokenize(candidate_normalized)
            for token in candidate_tokens:
                if len(token) >= n:
                    candidate_ngrams.add(f"TOKEN:{token}")
            
            overlap = len(candidate_ngrams.intersection(text_ngrams))
            token_bonus = 0
            if self.token_similarity:
                query_tokens = set(tokens)
                cand_tokens = set(candidate_tokens)
                if query_tokens and cand_tokens:
                    common = query_tokens.intersection(cand_tokens)
                    token_bonus = len(common) * 2
            candidate_scores[candidate] = overlap + token_bonus
        
        return sorted(candidate_scores.items(), key=lambda x: x[1], reverse=True)

    def _calculate_similarity(self, text: str, entity: str) -> float:
        """Calculate similarity between text and entity using multiple metrics."""
        normalized_text = self._normalize_text(text)
        normalized_entity = self._normalize_text(entity)
        
        if normalized_text == normalized_entity:
            return 100
        
        ratio_score = fuzz.ratio(normalized_text, normalized_entity)
        partial_score = 0
        if normalized_text in normalized_entity:
            partial_score = (len(normalized_text) / len(normalized_entity)) * 100
        elif normalized_entity in normalized_text:
            partial_score = (len(normalized_entity) / len(normalized_text)) * 100
        
        token_score = 0
        if self.token_similarity:
            text_tokens = set(self._tokenize(normalized_text))
            entity_tokens = set(self._tokenize(normalized_entity))
            if text_tokens and entity_tokens:
                common_tokens = text_tokens.intersection(entity_tokens)
                all_tokens = text_tokens.union(entity_tokens)
                token_score = (len(common_tokens) / len(all_tokens)) * 100 if all_tokens else 0
        
        token_sort_score = fuzz.token_sort_ratio(normalized_text, normalized_entity)
        token_set_score = fuzz.token_set_ratio(normalized_text, normalized_entity)
        
        return (
            ratio_score * 0.3 +
            partial_score * 0.1 +
            token_score * 0.2 +
            token_sort_score * 0.2 +
            token_set_score * 0.2
        )

    def find_entities(self, text: str, max_candidates_per_span: int = 10) -> List[Dict]:
        """Find entity matches in text using fuzzy matching."""
        potential_spans = self._extract_korean_spans(text)
        matches = []
        
        for span_text, start, end in potential_spans:
            if len(span_text.strip()) < self.min_entity_length:
                continue
            candidates = self._get_candidates(span_text)
            if not candidates:
                continue
            top_candidates = [c[0] for c in candidates[:max_candidates_per_span]]
            
            for entity in top_candidates:
                score = self._calculate_similarity(span_text, entity)
                if score >= self.min_similarity:
                    matches.append({
                        'text': span_text,
                        'matched_entity': entity,
                        'score': score,
                        'start': start,
                        'end': end,
                        'data': self.entity_data.get(entity, {})
                    })
        
        matches.sort(key=lambda x: (x['start'], -x['score']))
        return self._resolve_overlapping_matches(matches)

    def _extract_korean_spans(self, text: str) -> List[Tuple[str, int, int]]:
        """Extract potential entity spans from mixed Korean and English text."""
        spans = []
        min_len = self.min_entity_length
        patterns = [
            r'[a-zA-Z]+[가-힣]+(?:\s+[가-힣가-힣a-zA-Z0-9]+)*',
            r'[a-zA-Z]+\s+[가-힣]+(?:\s+[가-힣가-힣a-zA-Z0-9]+)*',
            r'[a-zA-Z]+[가-힣]+(?:[0-9]+)?',
            r'[a-zA-Z]+[가-힣]+\s+[가-힣]+',
            r'[a-zA-Z]+[가-힣]+\s+[가-힣]+\s+[가-힣]+',
            r'[a-zA-Z가-힣]+(?:\s+[a-zA-Z가-힣]+){1,3}',
            r'\d+\s+[a-zA-Z]+',
            r'[a-zA-Z가-힣0-9]+(?:\s+[a-zA-Z가-힣0-9]+)*'
        ]
        
        for pattern in patterns:
            for match in re.finditer(pattern, text):
                if len(match.group(0)) >= min_len:
                    spans.append((match.group(0), match.start(), match.end()))
        
        for span in re.split(r'[,\.!?;:"\'…\(\)\[\]\{\}\s_/]+', text):
            if span and len(span) >= min_len:
                span_pos = text.find(span)
                if span_pos != -1:
                    spans.append((span, span_pos, span_pos + len(span)))
        
        return spans

    def _resolve_overlapping_matches(self, matches: List[Dict], high_score_threshold: int = 50, overlap_tolerance: float = 0.5) -> List[Dict]:
        """Resolve overlapping matches by keeping the best match."""
        if not matches:
            return []
        
        sorted_matches = sorted(matches, key=lambda x: (-x['score'], x['end'] - x['start']))
        final_matches = []
        
        for current_match in sorted_matches:
            current_score = current_match['score']
            current_start, current_end = current_match['start'], current_match['end']
            current_range = set(range(current_start, current_end))
            current_len = len(current_range)
            current_match['overlap_ratio'] = 0.0
            
            if current_score >= high_score_threshold:
                is_too_similar = False
                for existing_match in final_matches:
                    if existing_match['score'] < high_score_threshold:
                        continue
                    existing_range = set(range(existing_match['start'], existing_match['end']))
                    intersection = current_range.intersection(existing_range)
                    current_overlap_ratio = len(intersection) / current_len if current_len > 0 else 0
                    current_match['overlap_ratio'] = max(current_match['overlap_ratio'], current_overlap_ratio)
                    
                    if current_overlap_ratio > overlap_tolerance and current_match['matched_entity'] == existing_match['matched_entity']:
                        is_too_similar = True
                        break
                if not is_too_similar:
                    final_matches.append(current_match)
            else:
                should_add = True
                for existing_match in final_matches:
                    existing_range = set(range(existing_match['start'], existing_match['end']))
                    intersection = current_range.intersection(existing_range)
                    current_overlap_ratio = len(intersection) / current_len if current_len > 0 else 0
                    current_match['overlap_ratio'] = max(current_match['overlap_ratio'], current_overlap_ratio)
                    if current_overlap_ratio > (1 - overlap_tolerance):
                        should_add = False
                        break
                if should_add:
                    final_matches.append(current_match)
        
        return sorted(final_matches, key=lambda x: x['start'])

def find_entities_in_text(
    text: str,
    entity_list: List[Union[str, Tuple[str, Dict]]],
    min_similarity: int = 70,
    ngram_size: int = 3,
    min_entity_length: int = 2,
    token_similarity: bool = True,
    high_score_threshold: int = 50,
    overlap_tolerance: float = 0.5
) -> List[Dict]:
    """Find entity matches in text using fuzzy matching."""
    matcher = KoreanEntityMatcher(
        min_similarity=min_similarity,
        ngram_size=ngram_size,
        min_entity_length=min_entity_length,
        token_similarity=token_similarity
    )
    matcher.build_from_list(entity_list)
    matches = matcher.find_entities(text)
    return matcher._resolve_overlapping_matches(
        matches, high_score_threshold=high_score_threshold, overlap_tolerance=overlap_tolerance
    )

def highlight_entities(text: str, matches: List[Dict]) -> str:
    """Highlight matched entities in text."""
    marked_text = text
    offset = 0
    for match in sorted(matches, key=lambda x: x['start'], reverse=True):
        start = match['start'] + offset
        end = match['end'] + offset
        entity = match['matched_entity']
        score = match['score']
        marked_text = marked_text[:start] + f"[{marked_text[start:end]}→{entity} ({score:.1f}%)]" + marked_text[end:]
        offset += len(f"[→{entity} ({score:.1f}%)]") + 2
    return marked_text

# Advanced Similarity Analysis
def advanced_sequential_similarity(str1: str, str2: str, metrics: List[str] = None, visualize: bool = False) -> Dict[str, float]:
    """Calculate multiple character-level similarity metrics between two strings."""
    if metrics is None:
        metrics = ['ngram', 'lcs', 'subsequence', 'difflib']
    results = {}
    
    if not str1 or not str2:
        return {metric: 0.0 for metric in metrics}
    
    s1, s2 = str1.lower(), str2.lower()
    
    if 'ngram' in metrics:
        ngram_scores = {}
        for window in range(min(len(s1), len(s2), 2), min(5, max(len(s1), len(s2)) + 1)):
            if len(s1) < window or len(s2) < window:
                ngram_scores[f'window_{window}'] = 0.0
                continue
            ngrams1 = [s1[i:i+window] for i in range(len(s1) - window + 1)]
            ngrams2 = [s2[i:i+window] for i in range(len(s2) - window + 1)]
            matches = sum(1 for ng in ngrams1 if ng in ngrams2)
            max_possible = max(len(ngrams1), len(ngrams2))
            score = matches / max_possible if max_possible > 0 else 0.0
            ngram_scores[f'window_{window}'] = score
        results['ngram'] = max(ngram_scores.values())
        results['ngram_details'] = ngram_scores
    
    if 'lcs' in metrics:
        def longest_common_substring(s1: str, s2: str) -> int:
            m, n = len(s1), len(s2)
            dp = [[0] * (n + 1) for _ in range(m + 1)]
            max_length = 0
            for i in range(1, m + 1):
                for j in range(1, n + 1):
                    if s1[i-1] == s2[j-1]:
                        dp[i][j] = dp[i-1][j-1] + 1
                        max_length = max(max_length, dp[i][j])
            return max_length
        
        lcs_length = longest_common_substring(s1, s2)
        max_length = max(len(s1), len(s2))
        results['lcs'] = lcs_length / max_length if max_length > 0 else 0.0
    
    if 'subsequence' in metrics:
        def longest_common_subsequence(s1: str, s2: str) -> int:
            m, n = len(s1), len(s2)
            dp = [[0] * (n + 1) for _ in range(m + 1)]
            for i in range(1, m + 1):
                for j in range(1, n + 1):
                    if s1[i-1] == s2[j-1]:
                        dp[i][j] = dp[i-1][j-1] + 1
                    else:
                        dp[i][j] = max(dp[i-1][j], dp[i][j-1])
            return dp[m][n]
        
        subseq_length = longest_common_subsequence(s1, s2)
        max_length = max(len(s1), len(s2))
        results['subsequence'] = subseq_length / max_length if max_length > 0 else 0.0
    
    if 'difflib' in metrics:
        sm = SequenceMatcher(None, s1, s2)
        results['difflib'] = sm.ratio()
    
    if visualize:
        try:
            sm = SequenceMatcher(None, s1, s2)
            matches = sm.get_matching_blocks()
            fig, ax = plt.subplots(figsize=(10, 3))
            ax.barh(0, len(s1), height=0.4, left=0, color='lightgray', alpha=0.3)
            ax.barh(1, len(s2), height=0.4, left=0, color='lightgray', alpha=0.3)
            
            for match in matches:
                i, j, size = match
                if size > 0:
                    ax.barh(0, size, height=0.4, left=i, color='green', alpha=0.5)
                    ax.barh(1, size, height=0.4, left=j, color='green', alpha=0.5)
                    ax.plot([i + size/2, j + size/2], [0.2, 0.8], 'k-', alpha=0.3)
            
            for i, c in enumerate(s1):
                ax.text(i + 0.5, 0, c, ha='center', va='center')
            for i, c in enumerate(s2):
                ax.text(i + 0.5, 1, c, ha='center', va='center')
            
            ax.set_yticks([0, 1])
            ax.set_yticklabels(['String 1', 'String 2'])
            ax.set_xlabel('Character Position')
            ax.set_title('Character-Level String Comparison')
            ax.grid(False)
            plt.tight_layout()
        except Exception as e:
            print(f"Visualization error: {e}")
    
    metrics_to_average = [m for m in results.keys() if not m.endswith('_details')]
    results['overall'] = sum(results[m] for m in metrics_to_average) / len(metrics_to_average)
    return results

In [ ]:
schema_prd = {
    "title": {
        "type": "string", 
        'description': '광고 제목. 광고의 핵심 주제와 가치 제안을 명확하게 설명할 수 있도록 생성'
    },
    'purpose': {
        'type': 'array', 
        'description': '광고의 주요 목적을 다음 중에서 선택(복수 가능): [상품 가입 유도, 대리점 방문 유도, 웹/앱 접속 유도, 이벤트 응모 유도, 혜택 안내, 쿠폰 제공 안내, 경품 제공 안내, 기타 정보 제공]'
    },
    'product': {
        'type': 'array',
        'items': {
            'type': 'object',
            'properties': {
            'name': {'type': 'string', 'description': '광고하는 제품이나 서비스 이름'},
            'action': {'type': 'string', 'description': '고객에게 기대하는 행동: [구매, 가입, 사용, 방문, 참여, 코드입력, 쿠폰다운로드, 기타] 중에서 선택'}
            }
        }
    },
    'channel': {
        'type': 'array', 
        'items': {
            'type': 'object', 
            'properties': {
                'type': {'type': 'string', 'description': '채널 종류: [URL, 전화번호, 앱, 대리점] 중에서 선택'},
                'value': {'type': 'string', 'description': '실제 URL, 전화번호, 앱 이름, 대리점 이름 등 구체적 정보'},
                'action': {'type': 'string', 'description': '채널 목적: [가입, 추가 정보, 문의, 수신, 수신 거부] 중에서 선택'},
                'benefit': {'type': 'string', 'description': '해당 채널 이용 시 특별 혜택'},
                'store_code': {'type': 'string', 'description': "매장 코드 - tworldfriends.co.kr URL에서 D+숫자 9자리(D[0-9]{9}) 패턴의 코드 추출하여 대리점 채널에 설정"}
            }
        }
    },
    'pgm':{
        'type': 'array', 
        'description': '아래 광고 분류 기준 정보에서 선택. 메세지 내용과 광고 분류 기준을 참고하여, 광고 메세지에 가장 부합하는 2개의 pgm_nm을 적합도 순서대로 제공'
    },
'required': ['purpose', 'product', 'channel', 'pgm'], 
'objectType': 'object'}

schema_prd_cot = {
"reasoning": {
    "type": "object",
    "description": "단계별 분석 과정 (최종 JSON에는 포함하지 않음)",
    "properties": {
    "step1_purpose_analysis": "광고 목적 분석 과정",
    "step2_product_identification": "상품 식별 및 도메인 매칭 과정", 
    "step3_channel_extraction": "채널 정보 추출 과정",
    "step4_pgm_classification": "프로그램 분류 과정"
    }
},
"title": {
    "type": "string",
    "description": "광고 제목. 광고의 핵심 주제와 가치 제안을 명확하게 설명"
},
"purpose": {
    "type": "array",
    "description": "STEP 1에서 분석한 광고의 주요 목적 (복수 가능). [상품 가입 유도, 대리점 방문 유도, 웹/앱 접속 유도, 이벤트 응모 유도, 혜택 안내, 쿠폰 제공 안내, 경품 제공 안내, 기타 정보 제공]에서 선택"
},
"product": {
    "type": "array",
    "items": {
    "type": "object",
    "properties": {
        "name": {
        "type": "string", 
        "description": "STEP 2에서 식별한 제품/서비스 이름"
        },
        "action": {
        "type": "string",
        "description": "STEP 2-3에서 결정한 고객 기대 행동. [구매, 가입, 사용, 방문, 참여, 코드입력, 쿠폰다운로드, 기타] 중에서 선택"
        },
        "domain": {
        "type": "string",
        "description": "매칭된 상품 후보의 도메인 정보 (참고용, 최종 JSON에는 제외)"
        }
    }
    }
},
"channel": {
    "type": "array",
    "items": {
      "type": "object",
      "properties": {
        "type": {
          "type": "string",
          "description": "채널 종류: [URL, 전화번호, 앱, 대리점] 중에서 선택"
        },
        "value": {
          "type": "string",
          "description": "실제 URL, 전화번호, 앱 이름, 대리점 이름 등 구체적 정보"
        },
        "action": {
          "type": "string",
          "description": "채널 목적: [가입, 추가 정보, 문의, 수신, 수신 거부] 중에서 선택"
        },
        "benefit": {
          "type": "string",
          "description": "해당 채널 이용 시 특별 혜택"
        },
        "store_code": {
          "type": "string",
          "description": "매장 코드 - tworldfriends.co.kr URL에서 D+숫자 9자리(D[0-9]{9}) 패턴의 코드 추출하여 대리점 채널에 설정"
        }
      }
    }
  },
"pgm": {
    "type": "array",
    "description": "STEP 4에서 선택한 프로그램 분류 (적합도 순 2개)"
}
}


In [ ]:
final_result_list = []
interim_result_list = []

for mms_msg in mms_pdf.sample(100)['msg'].tolist():

    final_result_dict = {}
    interim_result_dict = {}

    print(mms_msg)

    mms_embedding = model.encode([mms_msg.lower()], convert_to_tensor=True)

    similarities = torch.nn.functional.cosine_similarity(
        mms_embedding,  
        clue_embeddings,  
        dim=1 
    ).cpu().numpy()

    pgm_pdf_tmp = pgm_pdf.copy()
    pgm_pdf_tmp['sim'] = similarities

    pgm_pdf_tmp = pgm_pdf_tmp.sort_values('sim', ascending=False)

    def filter_specific_terms(strings: List[str]) -> List[str]:
        unique_strings = list(set(strings))  # 중복 제거
        unique_strings.sort(key=len, reverse=True)  # 길이 기준 내림차순 정렬

        filtered = []
        for s in unique_strings:
            if not any(s in other for other in filtered):
                filtered.append(s)

        return filtered

    def sliding_window_with_step(data, window_size, step=1):
        """Sliding window with configurable step size."""
        return [data[i:i + window_size] for i in range(0, len(data) - window_size + 1, step)]

    # tdf = pd.DataFrame([{'form_text':d[0],'tag_text':d[1],'start_text':d[2],'end_text':d[2]+d[3]} for d in kiwi_raw.analyze(mms_msg)[0][0]])
    result_msg_raw = kiwi_raw.tokenize(mms_msg, normalize_coda=True, z_coda=False, split_complex=False)
    token_list_msg = [d for d in result_msg_raw 
                    if d[1] not in tags_to_exclude
                    ]

    result_msg = kiwi.tokenize(mms_msg, normalize_coda=True, z_coda=False, split_complex=False)
    entities_from_kiwi = []
    for token in result_msg:  # 첫 번째 분석 결과의 토큰 리스트
        if token.tag == 'NNP' and token.form not in stop_item_names+['-'] and len(token.form)>=2 and not token.form.lower() in stop_item_names:  # 고유명사인 경우
        # if token.tag == 'NNG' and token.form in stop_item_names_ext:  # 고유명사인 경우
            entities_from_kiwi.append(token.form)

    from typing import List
    # 결과
    entities_from_kiwi = filter_specific_terms(entities_from_kiwi)

    # print("추출된 개체명:", list(set(entities_from_kiwi)))

    ngram_list_msg = []
    for w_size in range(1,5):
        windows = sliding_window_with_step(token_list_msg, w_size, step=1)
        windows_new = []
        for w in windows:
            tag_str = ','.join([t.tag for t in w])
            flag = True
            for et in exc_tag_patterns:
                if ','.join(et) in tag_str:
                    flag = False
                    # print(w)
                    break
        
            if flag:
                windows_new.append([[d.form for d in w], [d.tag for d in w]])

        ngram_list_msg.extend(windows_new)

    # 패턴에 해당하는 토큰 인덱스 찾기
    def find_pattern_indices(tokens, patterns):
        indices_to_exclude = set()
        
        # 단일 태그 패턴 먼저 체크
        for i in range(len(tokens)):
            for pattern in patterns:
                if len(pattern) == 1 and tokens[i].tag == pattern[0]:
                    indices_to_exclude.add(i)
        
        # 연속된 패턴 검사
        i = 0
        while i < len(tokens):
            if i in indices_to_exclude:
                i += 1
                continue
                
            for pattern in patterns:
                if len(pattern) > 1:  # 두 개 이상의 태그로 구성된 패턴
                    if i + len(pattern) <= len(tokens):  # 패턴 길이만큼 토큰이 남아있는지 확인
                        match = True
                        for j in range(len(pattern)):
                            if tokens[i+j].tag != pattern[j]:
                                match = False
                                break
                        
                        if match:  # 패턴이 일치하면 해당 토큰들의 인덱스를 모두 추가
                            for j in range(len(pattern)):
                                indices_to_exclude.add(i+j)
            i += 1
                        
        return indices_to_exclude

    # 패턴에 해당하지 않는 토큰만 필터링
    def filter_tokens_by_patterns(tokens, patterns):
        indices_to_exclude = find_pattern_indices(tokens, patterns)
        return [tokens[i] for i in range(len(tokens)) if i not in indices_to_exclude]

    # 제외된 토큰 없이 텍스트 재구성 - 단순 연결 방식
    def reconstruct_text_without_spaces(tokens):
        # 토큰들을 원래 시작 위치에 따라 정렬
        sorted_tokens = sorted(tokens, key=lambda token: token.start)
        
        result = []
        for token in sorted_tokens:
            result.append(token.form)
        
        # 토큰들을 공백 하나로 구분하여 결합
        return ' '.join(result)

    # 더 자연스러운 텍스트 재구성 - 원본 위치 기반 보존, 제외된 토큰은 건너뜀
    def reconstruct_text_preserved_positions(original_tokens, filtered_tokens):
        # 원본 토큰의 위치와 형태를 기록할 사전 생성
        token_map = {}
        for i, token in enumerate(original_tokens):
            token_map[(token.start, token.len)] = (i, token.form)
        
        # 필터링된 토큰의 인덱스 찾기
        filtered_indices = set()
        for token in filtered_tokens:
            key = (token.start, token.len)
            if key in token_map:
                filtered_indices.add(token_map[key][0])
        
        # 원본 순서대로 필터링된 토큰만 선택
        result = []
        for i, token in enumerate(original_tokens):
            if i in filtered_indices:
                result.append(token.form)
        
        return ' '.join(result)

    # 결과 출력
    filtered_tokens = filter_tokens_by_patterns(result_msg_raw, exc_tag_patterns)
    msg_text_filtered = reconstruct_text_preserved_positions(result_msg_raw, filtered_tokens)

    msg_text_filtered

    ngram_list_msg_filtered = []
    for w_size in range(2,4):
        windows = sliding_window_with_step(list(msg_text_filtered.lower().replace(' ', '')), w_size, step=1)
        ngram_list_msg_filtered.extend(windows)

    col_for_form_tmp_ent = 'char_entity'
    col_for_form_tmp_msg = 'char_msg'

    edf['form_tmp'] = edf[col_for_form_tmp_ent].apply(lambda x: [' '.join(s) for s in sliding_window_with_step(x, 2, step=1)])

    tdf = pd.DataFrame(ngram_list_msg).rename(columns={0:'token_txt', 1:'token_tag'})
    tdf['token_key'] = tdf.apply(lambda x: ''.join(x['token_txt'])+''.join(x['token_tag']), axis=1)
    tdf = tdf.drop_duplicates(['token_key']).drop(['token_key'], axis=1)
    tdf['char_msg'] = tdf.apply(lambda x: list((" ".join(x['token_txt'])).lower().replace(' ', '')), axis=1)

    tdf['form_tmp'] = tdf[col_for_form_tmp_msg].apply(lambda x: [' '.join(s) for s in sliding_window_with_step(x, 2, step=1)])
    tdf['token_txt_str'] = tdf['token_txt'].str.join(',')
    tdf['token_tag_str'] = tdf['token_tag'].str.join(',')

    # tdf['txt'] = tdf.apply(lambda x: ' '.join(x['token_txt']), axis=1) 

    fdf = edf.explode('form_tmp').merge(tdf.explode('form_tmp'), on='form_tmp').drop(['form_tmp'], axis=1)

    fdf = fdf.query("item_nm_alias.str.lower() not in @stop_item_names and token_txt_str.replace(',','').str.lower() not in @stop_item_names").drop_duplicates(['item_nm','item_nm_alias','item_id','token_txt_str','token_tag_str'])

    def ngram_jaccard_similarity(list1, list2, n=2):
        """Calculate similarity using Jaccard similarity of n-grams."""
        # Generate n-grams for both lists
        def get_ngrams(lst, n):
            return [tuple(lst[i:i+n]) for i in range(len(lst)-n+1)]
        
        # Handle edge cases
        if len(list1) < n or len(list2) < n:
            if list1 == list2:
                return 1.0
            else:
                return 0.0
        
        # Generate n-grams and calculate Jaccard similarity
        ngrams1 = set(get_ngrams(list1, n))
        ngrams2 = set(get_ngrams(list2, n))
        
        intersection = ngrams1.intersection(ngrams2)
        union = ngrams1.union(ngrams2)
        
        return len(intersection) / len(union) if union else 0

    def needleman_wunsch_similarity(list1, list2, match_score=1, mismatch_penalty=1, gap_penalty=1):
        """Global sequence alignment with Needleman-Wunsch algorithm."""
        m, n = len(list1), len(list2)
        
        # Initialize score matrix
        score = np.zeros((m+1, n+1))
        
        # Initialize first row and column with gap penalties
        for i in range(m+1):
            score[i][0] = -i * gap_penalty
        for j in range(n+1):
            score[0][j] = -j * gap_penalty
        
        # Fill the score matrix
        for i in range(1, m+1):
            for j in range(1, n+1):
                match = score[i-1][j-1] + (match_score if list1[i-1] == list2[j-1] else -mismatch_penalty)
                delete = score[i-1][j] - gap_penalty
                insert = score[i][j-1] - gap_penalty
                score[i][j] = max(match, delete, insert)
        
        # Calculate similarity score
        max_possible_score = min(m, n) * match_score
        alignment_score = score[m][n]
        
        # Normalize to 0-1 range
        min_possible_score = -max(m, n) * max(gap_penalty, mismatch_penalty)
        normalized_score = (alignment_score - min_possible_score) / (max_possible_score - min_possible_score)
        
        return normalized_score

    fdf['sim_score_token'] = fdf.apply(lambda row: needleman_wunsch_similarity(row['token_txt'], row['token_entity']), axis=1)
    fdf['sim_score_char'] = fdf.apply(lambda row: advanced_sequential_similarity((''.join(row['char_msg'])), (''.join(row['char_entity'])),metrics='difflib')['difflib'], axis=1)

    entity_list = [e.replace(' ', '').lower() for e in list(edf['item_nm_alias'].unique())]
    entities_from_kiwi_rev = [e.replace(' ', '').lower() for e in entities_from_kiwi]

    kdf = fdf.query("item_nm_alias in @entities_from_kiwi_rev or token_txt_str.str.replace(',',' ').str.lower() in @entities_from_kiwi_rev or token_txt_str.str.replace(',','').str.lower() in @entities_from_kiwi_rev").copy()
    kdf = kdf.query("(sim_score_token>=0.75 and sim_score_char>=0.75) or sim_score_char>=1").query("item_nm_alias.str.replace(',','').str.lower() in @entity_list or item_nm_alias.str.replace(' ','').str.lower() in @entity_list")
    kdf['rank'] = kdf.groupby(['token_txt_str'])['sim_score_char'].rank(ascending=False, method='dense')#.reset_index(name='rank')
    kdf = kdf.query("rank<=1")[['item_nm','item_nm_alias','item_id','token_txt_str','domain','sim_score_token','sim_score_char']].drop_duplicates()
    # kdf = kdf.query("rank<=1")

    # kdf = kdf.groupby('item_nm_alias', group_keys=False).apply(lambda x: x.sample(n=min(len(x), 2), random_state=42), include_groups=False)

    tags_to_exclude_final = ['SN']
    filtering_condition = [
    """not token_tag_str in @tags_to_exclude_final"""
    ,"""and token_txt_str.str.len()>=2"""
    ,"""and not token_txt_str in @stop_item_names"""
    ,"""and not token_txt_str.str.replace(',','').str.lower() in @stop_item_names"""
    ,"""and not item_nm_alias in @stop_item_names"""
    ]

    sdf = (
        fdf
        .query("item_nm_alias.str.lower() not in @stop_item_names")
        .query("(sim_score_token>=0.7 and sim_score_char>=0.8) or (sim_score_token>=0.1 and sim_score_char>=0.9)")
        # .query("item_nm_alias.str.contains('에이닷', case=False)")
        .query(' '.join(filtering_condition))
        .sort_values('sim_score_char', ascending=False)
        [['item_nm_alias','item_id','token_txt','token_txt_str','sim_score_token','sim_score_char','domain']]
    ).copy()

    sdf['rank_e'] = sdf.groupby(['item_nm_alias'])['sim_score_char'].rank(ascending=False, method='dense')#.reset_index(name='rank')
    sdf['rank_t'] = sdf.groupby(['token_txt_str'])['sim_score_char'].rank(ascending=False, method='dense')#.reset_index(name='rank')
    sdf = sdf.query("rank_t<=1 and rank_e<=1")[['item_nm_alias','item_id','token_txt_str','domain']].drop_duplicates()

    # sdf = sdf.groupby('item_nm_alias', group_keys=False).apply(lambda x: x.sample(n=min(len(x), 2), random_state=42), include_groups=False)

    if pd.concat([kdf,sdf]).shape[0]<1:
        continue
    
    product_df = pd.concat([kdf,sdf]).drop_duplicates(['item_id','item_nm','item_nm_alias','domain']).groupby(["item_nm","item_nm_alias","item_id","domain"])['token_txt_str'].apply(list).reset_index(name='item_name_in_message').rename(columns={'item_nm':'item_name_in_voca'}).sort_values('item_name_in_voca')

    product_df['item_name_in_message'] = product_df['item_name_in_message'].apply(lambda x: ",".join(list(set([w.replace(',',' ') for w in x]))))

    product_df[['item_name_in_message','item_name_in_voca','item_id','domain']]#.query("item_name_in_voca.str.contains('netflix', case=False)").drop_duplicates(['item_name_in_voca']).sort_values('item_id')

    ### Entity-Assisted

    # product_info = (",\n".join(product_df.apply(lambda x: f'"item_name_in_msg":"{x['item_name_in_msg']}", "item_name_in_voca":"{x['item_name_in_voca']}", "item_id":"{x['item_id']}:, "action":고객에게 기대하는 행동. [구매, 가입, 사용, 방문, 참여, 코드입력, 쿠폰다운로드, 없음, 기타] 중에서 선택', axis=1).tolist()))
    # product_info = ", ".join(product_df['item_name_in_voca'].unique().tolist())
    product_info = ", ".join(product_df[['item_name_in_voca','domain']].apply(lambda x: x['item_name_in_voca']+"("+x['domain']+")", axis=1))

    # product_df = product_df.drop_duplicates(['item_name_in_message','item_name_in_voca'])
    # product_df = product_df.merge(product_df.groupby('item_name_in_message')['item_id'].size().reset_index(name='count').sort_values('count', ascending=False), on='item_name_in_message', how='left').query('count<=3')
    product_df = product_df[['item_name_in_voca','item_id','domain']].drop_duplicates()
    product_df['action'] = '고객에게 기대하는 행동. [구매, 가입, 사용, 방문, 참여, 코드입력, 쿠폰다운로드, 없음, 기타] 중에서 선택'

    # product_element = product_df.to_dict(orient='records') if product_df.shape[0]>0 else schema_prd['product']

    # pgm_cand_info = "\n\t".join(pgm_pdf_tmp.iloc[:num_cand_pgms][['pgm_nm','clue_tag']].apply(lambda x: re.sub(r'\[.*?\]', '', x['pgm_nm'])+" : "+x['clue_tag'], axis=1).to_list())
    # rag_context = f"\n### 광고 분류 기준 정보 ###\n\t{pgm_cand_info}" if num_cand_pgms>0 else ""

    # schema_prd_ent = {
    #     "title": {
    #         "type": "string", 
    #         'description': '광고 제목. 광고의 핵심 주제와 가치 제안을 명확하게 설명할 수 있도록 생성'
    #     },
    #     "purpose": {
    #         "type": "array", 
    #         'description': '광고의 주요 목적을 다음 중에서 선택(복수 가능): [상품 가입 유도, 대리점 방문 유도, 웹/앱 접속 유도, 이벤트 응모 유도, 혜택 안내, 쿠폰 제공 안내, 경품 제공 안내, 기타 정보 제공]'
    #     },
    #     "product": 
    #         product_element
    #     ,
    #     'channel': {
    #         'type': 'array', 
    #         'items': {
    #             'type': 'object', 
    #             'properties': {
    #                 'type': {'type': 'string', 'description': '채널 종류: [URL, 전화번호, 앱, 대리점] 중에서 선택'},
    #                 'value': {'type': 'string', 'description': '실제 URL, 전화번호, 앱 이름, 대리점 이름 등 구체적 정보'},
    #                 'action': {'type': 'string', 'description': '채널 목적: [방문, 접속, 가입, 추가 정보, 문의, 수신, 수신 거부] 중에서 선택'},
    #                 # 'benefit': {'type': 'string', 'description': '해당 채널 이용 시 특별 혜택'},
    #                 'store_code': {'type': 'string', 'description': "매장 코드 - tworldfriends.co.kr URL에서 D+숫자 9자리(D[0-9]{9}) 패턴의 코드 추출하여 대리점 채널에 설정"}
    #             }
    #         },
    #     },
    #     'pgm':{
    #         'type': 'array', 
    #         'description': '아래 광고 분류 기준 정보에서 선택. 메세지 내용과 광고 분류 기준을 참고하여, 광고 메세지에 가장 부합하는 2개의 pgm_nm을 적합도 순서대로 제공'
    #     }
    # }

    # # Improved extraction guidance
    # extraction_guide = """
    # ### 분석 목표 ###
    # * Schema의 Product 태그 내에 action을 추출하세요.
    # * Schema내 action 항목 외 태그 정보는 원본 그대로 두세요.

    # ### 고려사항 ###
    # * 상품 정보에 있는 항목을 임의로 변형하거나 누락시키지 마세요.
    # * 광고 분류 기준 정보는 pgm_nm : clue_tag 로 구성

    # ### JSON 응답 형식 ###
    # 응답은 설명 없이 순수한 JSON 형식으로만 제공하세요. 응답의 시작과 끝은 '{'와 '}'여야 합니다. 어떠한 추가 텍스트나 설명도 포함하지 마세요.
    # """

    # # Create the system message with clear JSON output requirements
    # user_message = f"""당신은 SKT 캠페인 메시지에서 정확한 정보를 추출하는 전문가입니다. 아래 schema에 따라 광고 메시지를 분석하여 완전하고 정확한 JSON 객체를 생성해 주세요:

    # ### 분석 대상 광고 메세지 ###
    # {mms_msg}

    # ### 결과 Schema ###
    # {json.dumps(schema_prd_ent, indent=2, ensure_ascii=False)}

    # {extraction_guide}

    # {rag_context}

    # """

    # try:
    #     # # Use OpenAI's ChatCompletion with the current API format
    #     # response = client.chat.completions.create(
    #     #     model="skt/a.x-3-lg",  # Or your preferred OpenAI model
    #     # # model="skt/claude-3-5-sonnet-20241022",
    #     #     messages = [
    #     #         {"role": "user", "content": user_message},
    #     #     ],
    #     #     temperature=0.0,
    #     #     max_tokens=4000,
    #     #     top_p=0.95,  # Reduces randomness
    #     #     frequency_penalty=0.0,  # Avoid repetition in JSON
    #     #     presence_penalty=0.0,
    #     #     response_format={"type": "json_object"}  # Explicitly request JSON format
    #     # )
        
    #     # # Extract the JSON from the response
    #     # result_json_text = response.choices[0].message.content
    #     # json_objects = extract_json_objects(result_json_text)[0]

    #     llm_result = llm_gem3.invoke(user_message, max_tokens=4000)
    #     json_objects = extract_json_objects(llm_result.content)[0]

    #     pgm_json = pgm_pdf[pgm_pdf['pgm_nm'].apply(lambda x: re.sub(r'\[.*?\]', '', x) in ' '.join(json_objects['pgm']))][['pgm_nm','pgm_id']].to_dict('records')

    #     final_result = json_objects.copy()
    #     final_result['pgm'] = pgm_json

    #     final_result_dict['ent'] = final_result

    # except Exception as e:
    #     print(f"Error with API call: {e}")

    # print(json.dumps(final_json, indent=4, ensure_ascii=False))

    ### LLM-only

    extraction_guide = """
    ### 분석 시 고려사항 ###
    * 하나의 광고에 여러 상품이 포함될 수 있으며, 각 상품별로 별도 객체 생성
    * 재현율이 높도록 모든 상품을 선택
    * 상품 후보 정보는 상품 이름 (도메인) 형식으로 제공
    * 광고 분류 기준 정보는 pgm_nm : clue_tag 로 구성

    ### 분석 목표 ###
    * 텍스트 매칭 기법으로 만들어진 상품 후보 정보가 제공되면 이를 확인하여 참고하라.
    * 제공된 상품 이름이 적합하지 않으면 무시하고, 목록에 없어도 적합한 상품이 있으면 추출하세요.

    ### JSON 응답 형식 ###
    응답은 설명 없이 순수한 JSON 형식으로만 제공하세요. 응답의 시작과 끝은 '{'와 '}'여야 합니다. 어떠한 추가 텍스트나 설명도 포함하지 마세요.
    """

    # product_info = ", ".join(product_df['item_name_in_voca'].unique().tolist())
    product_info = ", ".join(product_df[['item_name_in_voca','domain']].apply(lambda x: x['item_name_in_voca']+"("+x['domain']+")", axis=1))

    rag_context = f"### 상품 후보 정보 ###\n\t{product_info}" if product_df.shape[0]>0 else ""

    pgm_cand_info = "\n\t".join(pgm_pdf_tmp.iloc[:num_cand_pgms][['pgm_nm','clue_tag']].apply(lambda x: re.sub(r'\[.*?\]', '', x['pgm_nm'])+" : "+x['clue_tag'], axis=1).to_list())
    rag_context += f"\n\n### 광고 분류 기준 정보 ###\n\t{pgm_cand_info}" if num_cand_pgms>0 else ""

    # Create the system message with clear JSON output requirements
    user_message = f"""당신은 SKT 캠페인 메시지에서 정확한 정보를 추출하는 전문가입니다. 아래 schema에 따라 광고 메시지를 분석하여 완전하고 정확한 JSON 객체를 생성해 주세요:

    ### 분석 대상 광고 메세지 ###
    {mms_msg}

    ### 결과 Schema ###
    {json.dumps(schema_prd, indent=2, ensure_ascii=False)}

    {extraction_guide}

    {rag_context}

    """

    try:
        # Use OpenAI's ChatCompletion with the current API format
        response = client.chat.completions.create(
            model="skt/a.x-3-lg",  # Or your preferred OpenAI model
        #   model="skt/claude-3-5-sonnet-20241022",
            messages = [
                {"role": "user", "content": user_message},
            ],
            temperature=0.0,
            max_tokens=4000,
            top_p=0.95,  # Reduces randomness
            frequency_penalty=0.0,  # Avoid repetition in JSON
            presence_penalty=0.0,
            response_format={"type": "json_object"}  # Explicitly request JSON format
        )
        
        # Extract the JSON from the response
        result_json_text = response.choices[0].message.content
        json_objects = extract_json_objects(result_json_text)[0]

        interim_result_dict['ax'] = json_objects

        llm_result = llm_gem3.invoke(user_message, max_tokens=4000)
        json_objects = extract_json_objects(llm_result.content)[0]

        interim_result_dict['gem'] = json_objects

    except Exception as e:
        print(f"Error with API call: {e}")
        # print(f"Error with API call: {e}")

    # matches = []
    # for item_name_message in json_objects['product']:
    #     matches.extend(find_entities_in_text(
    #         item_name_message['name'], 
    #         entity_list_for_fuzzy, 
    #         min_similarity=50,
    #         high_score_threshold=50,
    #         overlap_tolerance=0.5
    #     ))

    # mdf = pd.DataFrame(matches)
    # if len(matches)>0:
    #     mdf = mdf.query("text.str.lower() not in @stop_item_names and matched_entity.str.lower() not in @stop_item_names")

    # if mdf.shape[0]>0:
    #     mdf['item_id'] = mdf['data'].apply(lambda x: x['item_id'])
    #     mdf['domain'] = mdf['data'].apply(lambda x: x['domain'])
    #     mdf = mdf.query("not matched_entity.str.contains('test', case=False)").drop_duplicates(['item_id','domain'])

    #     mdf = mdf.merge(mdf.groupby(['text','start'])['end'].max().reset_index(name='end'), on=['text', 'start', 'end'])

    #     mdf['rank'] = mdf['data'].apply(lambda x: x['rank'])
    #     mdf['re_rank'] = mdf.groupby('text')['score'].rank(ascending=False)
    #     mdf = mdf.query("re_rank<=2")

    #     mdf = mdf.merge(pd.DataFrame(json_objects['product']).rename(columns={'name':'text'}), on='text', how='left')

    #     product_tag = mdf.rename(columns={'text':'item_name_in_message','matched_entity':'item_name_in_voca'})[['item_name_in_message','item_name_in_voca','item_id','domain']].drop_duplicates().to_dict(orient='records')

    #     final_result = {
    #         "title":json_objects['title'],
    #         "purpose":json_objects['purpose'],
    #         "product":product_tag,
    #         "channel":json_objects['channel'],
    #         "pgm":json_objects['pgm']
    #     }

    # else:
    #     final_result = json_objects
    #     final_result['product'] = [{'item_name_in_message':d['name'], 'item_name_in_voca':d['name'], 'item_id': '#', 'domain': '#'} for d in final_result['product']]

    # if num_cand_pgms>0:
    #     pgm_json = pgm_pdf[pgm_pdf['pgm_nm'].apply(lambda x: re.sub(r'\[.*?\]', '', x) in ' '.join(json_objects['pgm']))][['pgm_nm','pgm_id']].to_dict('records')
    #     final_result['pgm'] = pgm_json

    # # print(json.dumps(final_result, indent=4, ensure_ascii=False))

    # final_result_dict['llm'] = final_result

    ### cld 40

    try:
        response = llm_cld40.invoke([
                {"role": "user", "content": user_message}
        ])
        
        # Extract the JSON from the response
        result_json_text = response.content
        json_objects = extract_json_objects(result_json_text)[0]

        interim_result_dict['c40'] = json_objects

    except Exception as e:
        print(f"Error with API call: {e}")

    # matches = []
    # for item_name_message in json_objects['product']:
    #     matches.extend(find_entities_in_text(
    #         item_name_message['name'], 
    #         entity_list_for_fuzzy, 
    #         min_similarity=50,
    #         high_score_threshold=50,
    #         overlap_tolerance=0.5
    #     ))

    # mdf = pd.DataFrame(matches)
    # if len(matches)>0:
    #     mdf = mdf.query("text.str.lower() not in @stop_item_names and matched_entity.str.lower() not in @stop_item_names")

    # if mdf.shape[0]>0:
    #     mdf['item_id'] = mdf['data'].apply(lambda x: x['item_id'])
    #     mdf['domain'] = mdf['data'].apply(lambda x: x['domain'])
    #     mdf = mdf.query("not matched_entity.str.contains('test', case=False)").drop_duplicates(['item_id','domain'])

    #     mdf = mdf.merge(mdf.groupby(['text','start'])['end'].max().reset_index(name='end'), on=['text', 'start', 'end'])

    #     mdf['rank'] = mdf['data'].apply(lambda x: x['rank'])
    #     mdf['re_rank'] = mdf.groupby('text')['score'].rank(ascending=False)
    #     mdf = mdf.query("re_rank<=2")

    #     mdf = mdf.merge(pd.DataFrame(json_objects['product']).rename(columns={'name':'text'}), on='text', how='left')

    #     product_tag = mdf.rename(columns={'text':'item_name_in_message','matched_entity':'item_name_in_voca'})[['item_name_in_message','item_name_in_voca','item_id','domain']].drop_duplicates().to_dict(orient='records')

    #     final_result = {
    #         "title":json_objects['title'],
    #         "purpose":json_objects['purpose'],
    #         "product":product_tag,
    #         "channel":json_objects['channel'],
    #         "pgm":json_objects['pgm']
    #     }

    # else:
    #     final_result = json_objects
    #     final_result['product'] = [{'item_name_in_message':d['name'], 'item_name_in_voca':d['name'], 'item_id': '#', 'domain': '#'} for d in final_result['product']]

    # if num_cand_pgms>0:
    #     pgm_json = pgm_pdf[pgm_pdf['pgm_nm'].apply(lambda x: re.sub(r'\[.*?\]', '', x) in ' '.join(json_objects['pgm']))][['pgm_nm','pgm_id']].to_dict('records')
    #     final_result['pgm'] = pgm_json

    # # print(json.dumps(final_result, indent=4, ensure_ascii=False))

    # final_result_dict['c40'] = final_result

    ### LLM-COT

    # extraction_guide = """
    # ## 분석 지침
    # 1. **재현율 우선**: 광고에서 언급된 모든 상품을 누락 없이 추출
    # 2. **도메인 활용**: 상품 후보의 도메인 정보를 적극 활용하여 정확한 매칭 수행
    # 3. **목적 기반 추론**: 광고 목적을 명확히 파악한 후 다른 요소들을 일관성 있게 분석
    # 4. **채널 완전성**: 모든 접촉 채널을 누락 없이 추출하고 각각의 역할과 혜택을 명확히 식별
    # 5. **컨텍스트 고려**: 제공된 상품 후보가 부적합하면 무시하고, 누락된 중요 상품이 있으면 추가
    # 6. **매장 코드 정확성**: tworldfriends.co.kr URL에서 정확한 패턴 매칭을 통해 매장 코드 추출

    # ## JSON 응답 형식
    # - reasoning 섹션은 분석 과정 설명용이며 최종 JSON에는 포함하지 않음
    # - 순수한 JSON 형식으로만 응답
    # - 시작과 끝은 '{'와 '}'
    # - 추가 텍스트나 설명 없이 JSON만 제공
    # """

    # # product_info = ", ".join(product_df['item_name_in_voca'].unique().tolist())
    # product_info = ", ".join(product_df[['item_name_in_voca','domain']].apply(lambda x: x['item_name_in_voca']+"("+x['domain']+")", axis=1))

    # rag_context = f"### 상품 후보 정보 ###\n\t{product_info}" if product_df.shape[0]>0 else ""

    # pgm_cand_info = "\n\t".join(pgm_pdf_tmp.iloc[:num_cand_pgms][['pgm_nm','clue_tag']].apply(lambda x: re.sub(r'\[.*?\]', '', x['pgm_nm'])+" : "+x['clue_tag'], axis=1).to_list())
    # rag_context += f"\n\n### 광고 분류 기준 정보 ###\n\t{pgm_cand_info}" if num_cand_pgms>0 else ""

    # # Create the system message with clear JSON output requirements
    # user_message = f"""당당신은 SKT 캠페인 메시지에서 정확한 정보를 추출하는 전문가입니다. **단계별 사고 과정(Chain of Thought)**을 통해 광고 메시지를 분석하여 완전하고 정확한 JSON 객체를 생성해 주세요.

    # ## 분석 단계 (Chain of Thought)

    # ### STEP 1: 광고 목적(Purpose) 분석
    # 먼저 광고 메시지 전체를 읽고 광고의 주요 목적을 파악하세요.

    # ### STEP 2: 상품(Product) 식별 및 도메인 매칭
    # 파악된 목적을 바탕으로 다음 과정을 거쳐 상품을 식별하세요:

    # **2-1. 광고 메시지에서 언급된 모든 상품/서비스 추출**
    # - 직접적으로 언급된 상품명을 모두 나열
    # - 묵시적으로 언급된 서비스나 혜택도 포함

    # **2-2. RAG Context의 상품 후보 정보와 도메인 매칭**
    # - 각 추출된 상품을 상품 후보 정보와 비교
    # - 도메인 정보(product, subscription_service 등)를 고려하여 가장 적합한 매칭 수행
    # - 상품 후보에 없어도 광고에서 중요하게 다뤄지는 상품이 있다면 추가

    # **2-3. 각 상품별 고객 행동(Action) 결정**
    # - STEP 1에서 파악한 목적과 연결하여 각 상품에 대한 기대 행동 결정
    # - 행동 후보: [구매, 가입, 사용, 방문, 참여, 코드입력, 쿠폰다운로드, 기타]

    # ### STEP 3: 채널(Channel) 및 연락처 정보 추출
    # 광고 메시지에서 고객이 접촉할 수 있는 모든 채널을 식별하고 분석하세요:

    # **3-1. 채널 유형별 식별**
    # - **URL**: 웹사이트 링크, 프로모션 페이지, 랜딩 페이지 등
    # - **전화번호**: 고객센터, 상담 전화, 수신거부 번호 등
    # - **앱**: 모바일 앱, 웹앱 등의 애플리케이션
    # - **대리점**: 매장, 지점, 서비스센터 등

    # **3-2. 각 채널별 세부 정보 분석**
    # - **value**: 정확한 URL, 전화번호, 앱명, 대리점명 추출
    # - **action**: 해당 채널의 주요 목적 파악 [가입, 추가 정보, 문의, 수신, 수신 거부]
    # - **benefit**: 채널 이용 시 제공되는 특별 혜택이나 무료 서비스 등
    # - **store_code**: tworldfriends.co.kr URL에서 D+숫자 9자리(D[0-9]{9}) 패턴 추출

    # **3-3. 채널 우선순위 및 역할 분석**
    # - 주요 행동 유도 채널 vs 보조 정보 제공 채널 구분
    # - 각 채널이 STEP 1에서 파악한 목적과 어떻게 연결되는지 분석

    # ### STEP 4: 프로그램 분류(PGM) 결정
    # - 광고 분류 기준 정보의 키워드와 메시지 내용 매칭
    # - 적합도 순서대로 2개 선택

    # ### 분석 대상 광고 메세지 ###
    # {mms_msg}

    # {rag_context}

    # ### 결과 Schema ###
    # {json.dumps(schema_prd_cot, indent=2, ensure_ascii=False)}

    # {extraction_guide}

    # """

    # try:
    #     # Use OpenAI's ChatCompletion with the current API format
    #     response = client.chat.completions.create(
    #         # model="skt/a.x-3-lg",  # Or your preferred OpenAI model
    #       model="skt/claude-3-5-sonnet-20241022",
    #         messages = [
    #             {"role": "user", "content": user_message},
    #         ],
    #         temperature=0.0,
    #         max_tokens=4000,
    #         top_p=0.95,  # Reduces randomness
    #         frequency_penalty=0.0,  # Avoid repetition in JSON
    #         presence_penalty=0.0,
    #         response_format={"type": "json_object"}  # Explicitly request JSON format
    #     )
        
    #     # Extract the JSON from the response
    #     result_json_text = response.choices[0].message.content
    #     json_objects = extract_json_objects(result_json_text)[0]

    #     interim_result_dict['cot'] = json_objects
                    
    # except Exception as e:
    #     print(f"Error with API call: {e}")

    # matches = []
    # for item_name_message in json_objects['product']:
    #     matches.extend(find_entities_in_text(
    #         item_name_message['name'], 
    #         entity_list_for_fuzzy, 
    #         min_similarity=50,
    #         high_score_threshold=50,
    #         overlap_tolerance=0.5
    #     ))

    # mdf = pd.DataFrame(matches)
    # if len(matches)>0:
    #     mdf = mdf.query("text.str.lower() not in @stop_item_names and matched_entity.str.lower() not in @stop_item_names")

    # if mdf.shape[0]>0:
    #     mdf['item_id'] = mdf['data'].apply(lambda x: x['item_id'])
    #     mdf['domain'] = mdf['data'].apply(lambda x: x['domain'])
    #     mdf = mdf.query("not matched_entity.str.contains('test', case=False)").drop_duplicates(['item_id','domain'])

    #     mdf = mdf.merge(mdf.groupby(['text','start'])['end'].max().reset_index(name='end'), on=['text', 'start', 'end'])

    #     mdf['rank'] = mdf['data'].apply(lambda x: x['rank'])
    #     mdf['re_rank'] = mdf.groupby('text')['score'].rank(ascending=False)
    #     mdf = mdf.query("re_rank<=2")

    #     mdf = mdf.merge(pd.DataFrame(json_objects['product']).rename(columns={'name':'text'}), on='text', how='left')

    #     product_tag = mdf.rename(columns={'text':'item_name_in_message','matched_entity':'item_name_in_voca'})[['item_name_in_message','item_name_in_voca','item_id','domain']].drop_duplicates().to_dict(orient='records')

    #     final_result = {
    #         "title":json_objects['title'],
    #         "purpose":json_objects['purpose'],
    #         "product":product_tag,
    #         "channel":json_objects['channel'],
    #         "pgm":json_objects['pgm']
    #     }

    # else:
    #     final_result = json_objects
    #     final_result['product'] = [{'item_name_in_message':d['name'], 'item_name_in_voca':d['name'], 'item_id': '#', 'domain': '#'} for d in final_result['product']]

    # if num_cand_pgms>0:
    #     pgm_json = pgm_pdf[pgm_pdf['pgm_nm'].apply(lambda x: re.sub(r'\[.*?\]', '', x) in ' '.join(json_objects['pgm']))][['pgm_nm','pgm_id']].to_dict('records')
    #     final_result['pgm'] = pgm_json

    # # print(json.dumps(final_result, indent=4, ensure_ascii=False))

    # final_result_dict['cot'] = final_result

    final_result_list.append(final_result_dict)
    interim_result_list.append(interim_result_dict)



In [ ]:
from difflib import SequenceMatcher
def calculate_list_similarity(list1, list2):
    """Calculate Jaccard similarity between two lists"""
    if isinstance(list1, dict):
        list1 = [str(item) for item in list1.values()]
    if isinstance(list2, dict):
        list2 = [str(item) for item in list2.values()]
    # Ensure lists contain strings
    list1 = [str(item) for item in list1]
    list2 = [str(item) for item in list2]
    # Convert lists to sets for comparison
    set1 = set(sorted(set(list1)))
    set2 = set(sorted(set(list2)))
    # Calculate Jaccard similarity
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union > 0 else 0
def calculate_text_similarity(text1, text2):
    """Calculate text similarity using SequenceMatcher"""
    return SequenceMatcher(None, str(text1), str(text2)).ratio()
def calculate_product_similarity(prod1, prod2):
    """Calculate similarity between product dictionaries with detailed structure"""
    if not isinstance(prod1, dict) or not isinstance(prod2, dict):
        return 0.0
    # Calculate similarity for each field
    item_name_message_sim = calculate_text_similarity(
        prod1.get('item_name_in_message', '#'),
        prod2.get('item_name_in_message', '&')
    )
    item_name_voca_sim = calculate_text_similarity(
        prod1.get('item_name_in_voca', '#'),
        prod2.get('item_name_in_voca', '&')
    )
    item_id_sim = calculate_text_similarity(
        prod1.get('item_id', '#'),
        prod2.get('item_id', '&')
    )
    domain_sim = calculate_text_similarity(
        prod1.get('domain', '#'),
        prod2.get('domain', '&')
    )
    name_sim = calculate_text_similarity(
        prod1.get('name', '#'),
        prod2.get('name', '&')
    )
    action_sim = calculate_text_similarity(
        prod1.get('action', '#'),
        prod2.get('action', '&')
    )
    # print(item_name_message_sim, item_name_voca_sim, item_id_sim, domain_sim, name_sim, action_sim)
    # Weighted average - item_id and domain are more distinctive
    similarity = (
        item_name_message_sim +
        item_name_voca_sim +
        item_id_sim +
        domain_sim +
        name_sim +
        action_sim
    )/len(prod1.keys())
    return similarity
def calculate_channel_similarity(chan1, chan2):
    """Calculate similarity between channel dictionaries"""
    if not isinstance(chan1, dict) or not isinstance(chan2, dict):
        return 0.0
    type_sim = calculate_text_similarity(chan1.get('type', ''), chan2.get('type', ''))
    value_sim = calculate_text_similarity(chan1.get('value', ''), chan2.get('value', ''))
    action_sim = calculate_text_similarity(chan1.get('action', ''), chan2.get('action', ''))
    return (type_sim + value_sim + action_sim) / 3
def calculate_pgm_similarity(pgm1, pgm2):
    """Calculate similarity between program dictionaries"""
    if isinstance(pgm1, dict) and isinstance(pgm2, dict):
        pgm_nm_sim = calculate_text_similarity(pgm1.get('pgm_nm', ''), pgm2.get('pgm_nm', ''))
        pgm_id_sim = calculate_text_similarity(pgm1.get('pgm_id', ''), pgm2.get('pgm_id', ''))
        pgm_sim = pgm_nm_sim * 0.4 + pgm_id_sim * 0.6
    else:
        pgm_sim = 0.0
    # pgm_id is more distinctive, so give it higher weight
    return pgm_sim
def calculate_products_list_similarity(products1, products2):
    """Calculate similarity between two lists of product dictionaries"""
    if not products1 or not products2:
        return 0.0
    # For each product in list1, find best match in list2
    similarities = []
    for p1 in products1:
        best_match = 0.0
        for p2 in products2:
            similarity = calculate_product_similarity(p1, p2)
            best_match = max(best_match, similarity)
        similarities.append(best_match)
    # Also check reverse direction to handle different list sizes
    reverse_similarities = []
    for p2 in products2:
        best_match = 0.0
        for p1 in products1:
            similarity = calculate_product_similarity(p1, p2)
            best_match = max(best_match, similarity)
        reverse_similarities.append(best_match)
    # Take average of both directions
    forward_avg = sum(similarities) / len(similarities)
    reverse_avg = sum(reverse_similarities) / len(reverse_similarities)
    return (forward_avg + reverse_avg) / 2
def calculate_channels_list_similarity(channels1, channels2):
    """Calculate similarity between two lists of channel dictionaries"""
    if not channels1 or not channels2:
        return 0.0
    similarities = []
    for c1 in channels1:
        best_match = 0.0
        for c2 in channels2:
            similarity = calculate_channel_similarity(c1, c2)
            best_match = max(best_match, similarity)
        similarities.append(best_match)
    return sum(similarities) / len(similarities)
def calculate_pgms_list_similarity(pgms1, pgms2):
    """Calculate similarity between two lists of program dictionaries"""
    if not pgms1 or not pgms2:
        return 0.0
    if isinstance(pgms1, list) and isinstance(pgms2, list):
        # print(pgms1, pgms2)
        pgm_sim = calculate_list_similarity(pgms1, pgms2)
        return pgm_sim
    # For each pgm in list1, find best match in list2
    similarities = []
    for p1 in pgms1:
        best_match = 0.0
        for p2 in pgms2:
            similarity = calculate_pgm_similarity(p1, p2)
            best_match = max(best_match, similarity)
        similarities.append(best_match)
    # Also check reverse direction
    reverse_similarities = []
    for p2 in pgms2:
        best_match = 0.0
        for p1 in pgms1:
            similarity = calculate_pgm_similarity(p1, p2)
            best_match = max(best_match, similarity)
        reverse_similarities.append(best_match)
    # Take average of both directions
    forward_avg = sum(similarities) / len(similarities)
    reverse_avg = sum(reverse_similarities) / len(reverse_similarities)
    return (forward_avg + reverse_avg) / 2
def calculate_dictionary_similarity(dict1, dict2):
    """
    Calculate similarity between two dictionaries with generalized structure:
    {
        'title': str,
        'purpose': [list of strings],
        'product': [list of product dicts],
        'channel': [list of channel dicts],
        'pgm': [list of program dicts]
    }
    """
    if not isinstance(dict1, dict) or not isinstance(dict2, dict):
        return {'overall_similarity': 0.0, 'error': 'Both inputs must be dictionaries'}
    # Calculate title similarity
    title_similarity = calculate_text_similarity(
        dict1.get('title', ''),
        dict2.get('title', '')
    )
    # Calculate purpose similarity (list of strings)
    purpose_similarity = calculate_list_similarity(
        dict1.get('purpose', []),
        dict2.get('purpose', [])
    )
    # Calculate product similarity (list of product dicts)
    product_similarity = calculate_products_list_similarity(
        dict1.get('product', []),
        dict2.get('product', [])
    )
    # Calculate channel similarity (list of channel dicts)
    channel_similarity = calculate_channels_list_similarity(
        dict1.get('channel', []),
        dict2.get('channel', [])
    )
    # Calculate pgm similarity (list of program dicts)
    pgm_similarity = calculate_pgms_list_similarity(
        dict1.get('pgm', []),
        dict2.get('pgm', [])
    )
    # Calculate overall similarity (weighted average)
    # Adjusted weights to reflect importance of each component
    overall_similarity = (
        title_similarity * 0.2 +
        purpose_similarity * 0.15 +
        product_similarity * 0.35 +
        channel_similarity * 0.15 +
        pgm_similarity * 0.15
    )
    return {
        'overall_similarity': overall_similarity,
        'title_similarity': title_similarity,
        'purpose_similarity': purpose_similarity,
        'product_similarity': product_similarity,
        'channel_similarity': channel_similarity,
        'pgm_similarity': pgm_similarity
    }
def get_detailed_product_comparison(dict1, dict2):
    """
    Get detailed comparison of products between two dictionaries
    """
    products1 = dict1.get('product', [])
    products2 = dict2.get('product', [])
    detailed_comparison = []
    for i, p1 in enumerate(products1):
        best_match = {'similarity': 0.0, 'match_index': -1, 'match_product': None}
        for j, p2 in enumerate(products2):
            similarity = calculate_product_similarity(p1, p2)
            if similarity > best_match['similarity']:
                best_match = {
                    'similarity': similarity,
                    'match_index': j,
                    'match_product': p2
                }
        detailed_comparison.append({
            'product1_index': i,
            'product1': p1,
            'best_match': best_match
        })
    return detailed_comparison
# Example usage
if __name__ == "__main__":
    # Example dictionaries with the new schema
    dict1 = {
        'title': 'POOQ 콘텐츠 팩 출시 기념 혜택',
        'purpose': ['상품 가입 유도'],
        'product': [
            {'item_name_in_message': 'POOQ', 'item_name_in_voca': 'PooQ 팩', 'item_id': 'T000009330', 'domain': 'product'},
            {'item_name_in_message': 'POOQ 콘텐츠 팩', 'item_name_in_voca': 'FLO 콘텐츠 팩', 'item_id': 'PR00000217', 'domain': 'subscription_service'}
        ],
        'channel': [{'type': 'URL', 'value': 'http://t-mms.kr/t.do?m=#61&u=http://m2.tworld.co.kr/jsp/op.jsp?p=w1026', 'action': '가입'}],
        'pgm': [
            {'pgm_nm': '[마케팅_Sales]상품및부가서비스가입유도_구독'},
            {'pgm_nm': '[마케팅_Sales]타사회선(가망)_win-back'}
        ]
    }
    dict2 = {
        'title': 'POOQ 콘텐츠 팩 특별 혜택',
        'purpose': ['상품 가입 유도', '프로모션'],
        'product': [
            {'item_name_in_message': 'POOQ', 'item_name_in_voca': 'PooQ 팩', 'item_id': 'T000009330', 'domain': 'product'},
            {'item_name_in_message': 'POOQ 콘텐츠 팩', 'item_name_in_voca': 'FLO 콘텐츠 팩 플러스', 'item_id': 'PR00000218', 'domain': 'subscription_service'}
        ],
        'channel': [{'type': 'URL', 'value': 'http://different-url.com', 'action': '가입'}],
        'pgm': [
            {'pgm_nm': '[마케팅_Sales]상품및부가서비스가입유도_구독'}
        ]
    }
    # Calculate similarity
    result = calculate_dictionary_similarity(dict1, dict2)
    print("Similarity Results:")
    for key, value in result.items():
        print(f"{key}: {value:.3f}")
    print("\nDetailed Product Comparison:")
    detailed = get_detailed_product_comparison(dict1, dict2)
    for comparison in detailed:
        print(f"Product {comparison['product1_index']}: {comparison['product1']['item_name_in_message']}")
        print(f"  Best match (similarity: {comparison['best_match']['similarity']:.3f}): {comparison['best_match']['match_product']['item_name_in_message'] if comparison['best_match']['match_product'] else 'None'}")

In [ ]:
interim_result_list

In [ ]:
ax_result_list = []
gem_result_list = []
for interim_result_dict in interim_result_list:
    try:
        if len(interim_result_dict['c40'])<1 or len(interim_result_dict['ax'])<1 or len(interim_result_dict['gem'])<1: continue
        ax_result_list.append(calculate_dictionary_similarity(interim_result_dict['c40'], interim_result_dict['ax']))
        gem_result_list.append(calculate_dictionary_similarity(interim_result_dict['c40'], interim_result_dict['gem']))
    except Exception as e:
        pass
len(ax_result_list), len(gem_result_list)

In [ ]:
pd.DataFrame(ax_result_list).mean().to_frame().rename(columns={0:'a.x-3'}).merge(pd.DataFrame(gem_result_list).mean().to_frame().rename(columns={0:'gemma-3'}), left_index=True, right_index=True).round(2)


In [ ]:
interim_result_dict['c40']

In [ ]:
interim_result_dict['cot']

In [ ]:
calculate_dictionary_similarity(interim_result_dict['c40'], interim_result_dict['cot'])

In [ ]:
import spacy

nlp = spacy.load("ko_core_news_sm")

msg_text_list = ["""
    광고 제목:[SK텔레콤] 2월 0 day 혜택 안내
    광고 내용:(광고)[SKT] 2월 0 day 혜택 안내__[2월 10일(토) 혜택]_만 13~34세 고객이라면_베어유 모든 강의 14일 무료 수강 쿠폰 드립니다!_(선착순 3만 명 증정)_▶ 자세히 보기: http://t-mms.kr/t.do?m=#61&s=24589&a=&u=https://bit.ly/3SfBjjc__■ 에이닷 X T 멤버십 시크릿코드 이벤트_에이닷 T 멤버십 쿠폰함에 ‘에이닷이빵쏜닷’을 입력해보세요!_뚜레쥬르 데일리우유식빵 무료 쿠폰을 드립니다._▶ 시크릿코드 입력하러 가기: https://bit.ly/3HCUhLM__■ 문의: SKT 고객센터(1558, 무료)_무료 수신거부 1504
    """,
    """
    광고 제목:통화 부가서비스를 패키지로 저렴하게!
    광고 내용:(광고)[SKT] 콜링플러스 이용 안내  #04 고객님, 안녕하세요. <콜링플러스>에 가입하고 콜키퍼, 컬러링, 통화가능통보플러스까지 총 3가지의 부가서비스를 패키지로 저렴하게 이용해보세요.  ■ 콜링플러스 - 이용요금: 월 1,650원, 부가세 포함 - 콜키퍼(550원), 컬러링(990원), 통화가능통보플러스(770원)를 저렴하게 이용할 수 있는 상품  ■ 콜링플러스 가입 방법 - T월드 앱: 오른쪽 위에 있는 돋보기를 눌러 콜링플러스 검색 > 가입  ▶ 콜링플러스 가입하기: http://t-mms.kr/t.do?m=#61&u=https://skt.sh/17tNH  ■ 유의 사항 - 콜링플러스에 가입하면 기존에 이용 중인 콜키퍼, 컬러링, 통화가능통보플러스 서비스는 자동으로 해지됩니다. - 기존에 구매한 컬러링 음원은 콜링플러스 가입 후에도 계속 이용할 수 있습니다.(시간대, 발신자별 설정 정보는 다시 설정해야 합니다.)  * 최근 다운로드한 음원은 보관함에서 무료로 재설정 가능(다운로드한 날로부터 1년 이내)   ■ 문의: SKT 고객센터(114)  SKT와 함께해주셔서 감사합니다.  무료 수신거부 1504\n    ', 
    """,
    """
    (광고)[SKT] 1월 0 day 혜택 안내_ _[1월 20일(토) 혜택]_만 13~34세 고객이라면 _CU에서 핫바 1,000원에 구매 하세요!_(선착순 1만 명 증정)_▶ 자세히 보기 : http://t-mms.kr/t.do?m=#61&s=24264&a=&u=https://bit.ly/3H2OHSs__■ 에이닷 X T 멤버십 구독캘린더 이벤트_0 day 일정을 에이닷 캘린더에 등록하고 혜택 날짜에 알림을 받아보세요! _알림 설정하면 추첨을 통해 [스타벅스 카페 라떼tall 모바일쿠폰]을 드립니다. _▶ 이벤트 참여하기 : https://bit.ly/3RVSojv_ _■ 문의: SKT 고객센터(1558, 무료)_무료 수신거부 1504
    """,
    """
    '[T 우주] 넷플릭스와 웨이브를 월 9,900원에! \n(광고)[SKT] 넷플릭스+웨이브 월 9,900원, 이게 되네! __#04 고객님,_넷플릭스와 웨이브 둘 다 보고 싶었지만, 가격 때문에 망설이셨다면 지금이 바로 기회! __오직 T 우주에서만, _2개월 동안 월 9,900원에 넷플릭스와 웨이브를 모두 즐기실 수 있습니다.__8월 31일까지만 드리는 혜택이니, 지금 바로 가입해 보세요! __■ 우주패스 Netflix 런칭 프로모션 _- 기간 : 2024년 8월 31일(토)까지_- 혜택 : 우주패스 Netflix(광고형 스탠다드)를 2개월 동안 월 9,900원에 이용 가능한 쿠폰 제공_▶ 프로모션 자세히 보기: http://t-mms.kr/jAs/#74__■ 우주패스 Netflix(월 12,000원)  _- 기본 혜택 : Netflix 광고형 스탠다드 멤버십_- 추가 혜택 : Wavve 콘텐츠 팩 _* 추가 요금을 내시면 Netflix 스탠다드와 프리미엄 멤버십 상품으로 가입 가능합니다.  __■ 유의 사항_-  프로모션 쿠폰은 1인당 1회 다운로드 가능합니다. _-  쿠폰 할인 기간이 끝나면 정상 이용금액으로 자동 결제 됩니다. __■ 문의: T 우주 고객센터 (1505, 무료)__나만의 구독 유니버스, T 우주 __무료 수신거부 1504'
    """,
    """
    광고 제목:[SK텔레콤] T건강습관 X AIA Vitality, 우리 가족의 든든한 보험!
    광고 내용:(광고)[SKT] 가족의 든든한 보험 (무배당)AIA Vitality 베스트핏 보장보험 안내  고객님, 안녕하세요. 4인 가족 표준생계비, 준비하고 계시나요? (무배당)AIA Vitality 베스트핏 보장보험(디지털 전용)으로 최대 20% 보험료 할인과 가족의 든든한 보험 보장까지 누려 보세요.   ▶ 자세히 보기: http://t-mms.kr/t.do?m=#61&u=https://bit.ly/36oWjgX  ■ AIA Vitality  혜택 - 매달 리워드 최대 12,000원 - 등급 업그레이드 시 특별 리워드 - T건강습관 제휴 할인 최대 40% ※ 제휴사별 할인 조건과 주간 미션 달성 혜택 등 자세한 내용은 AIA Vitality 사이트에서 확인하세요. ※ 이 광고는 AIA생명의 광고이며 SK텔레콤은 모집 행위를 하지 않습니다.  - 보험료 납입 기간 중 피보험자가 장해분류표 중 동일한 재해 또는 재해 이외의 동일한 원인으로 여러 신체 부위의 장해지급률을 더하여 50% 이상인 장해 상태가 된 경우 차회 이후의 보험료 납입 면제 - 사망보험금은 계약일(부활일/효력회복일)로부터 2년 안에 자살한 경우 보장하지 않음 - 일부 특약 갱신 시 보험료 인상 가능 - 기존 계약 해지 후 신계약 체결 시 보험인수 거절, 보험료 인상, 보장 내용 변경 가능 - 해약 환급금(또는 만기 시 보험금이나 사고보험금)에 기타 지급금을 합해 5천만 원까지(본 보험 회사 모든 상품 합산) 예금자 보호 - 계약 체결 전 상품 설명서 및 약관 참조 - 월 보험료 5,500원(부가세 포함)  * 생명보험협회 심의필 제2020-03026호(2020-09-22) COM-2020-09-32426  ■문의: 청약 관련(1600-0880)  무료 수신거부 1504    
    """
    ]

message_idx = 0

longer_text = msg_text_list[message_idx]

doc = nlp(longer_text)

print("=== Alternative 1: Extract All Nouns and Proper Nouns ===")
nouns = []
for token in doc:
    if token.pos_ in ["NOUN", "PROPN"] and not token.is_space and not token.is_punct:
        nouns.append(f"{token.text} ({token.pos_})")

for noun in nouns:
    print(f"  {noun}")

print("\n=== Alternative 2: Group Adjacent Nouns ===")
def extract_noun_groups(doc):
    noun_groups = []
    current_group = []
    
    for token in doc:
        if token.pos_ in ["NOUN", "PROPN"]:
            current_group.append(token.text)
        else:
            if current_group:
                noun_groups.append(" ".join(current_group))
                current_group = []
    
    # Don't forget the last group
    if current_group:
        noun_groups.append(" ".join(current_group))
    
    return noun_groups

noun_groups = extract_noun_groups(doc)
for group in noun_groups:
    print(f"  {group}")

print("\n=== Alternative 3: Extract Compound Nouns (연속된 명사) ===")
def extract_compound_nouns(doc):
    compounds = []
    current_compound = []
    
    for token in doc:
        # Include nouns, proper nouns, and some particles that connect nouns
        if token.pos_ in ["NOUN", "PROPN"] or (token.pos_ == "ADP" and token.text in ["의", "에서"]):
            current_compound.append(token.text)
        else:
            if len(current_compound) > 1:  # Only keep compounds with multiple parts
                compounds.append("".join(current_compound))
            current_compound = []
    
    if len(current_compound) > 1:
        compounds.append("".join(current_compound))
    
    return compounds

compounds = extract_compound_nouns(doc)
for compound in compounds:
    print(f"  {compound}")

print("\n=== Alternative 4: Named Entities (Most Reliable) ===")
for ent in doc.ents:
    print(f"  {ent.text} ({ent.label_})")

print("\n=== Alternative 5: Custom Korean Noun Phrase Pattern ===")
# This is a simple heuristic for Korean noun phrases
def korean_noun_phrases(doc):
    phrases = []
    i = 0
    while i < len(doc):
        if doc[i].pos_ in ["NOUN", "PROPN"]:
            phrase = [doc[i].text]
            j = i + 1
            
            # Look ahead for particles and more nouns
            while j < len(doc):
                if doc[j].pos_ == "ADP" and doc[j].text in ["의", "에서", "에게", "로", "으로"]:
                    phrase.append(doc[j].text)
                    j += 1
                elif doc[j].pos_ in ["NOUN", "PROPN"]:
                    phrase.append(doc[j].text)
                    j += 1
                else:
                    break
            
            if len(phrase) > 1:
                phrases.append("".join(phrase))
            i = j
        else:
            i += 1
    
    return phrases

korean_phrases = korean_noun_phrases(doc)
for phrase in korean_phrases:
    print(f"  {phrase}")